In [1]:
import argparse

In [2]:
parser = argparse.ArgumentParser(description='Process some interers.')

In [3]:
parser

ArgumentParser(prog='ipykernel_launcher.py', usage=None, description='Process some interers.', formatter_class=<class 'argparse.HelpFormatter'>, conflict_handler='error', add_help=True)

`testlodaerIn == testlodaer10 == Cifar-10`

`length == 10000 && batch_size == 1, image 한장과 정답레이블 한개가 같이 내재`

In [19]:
for j, data in enumerate(testloaderIn):
    display(j)
    display(type(data))
    break

0

list

In [54]:
import torch
from torchvision import datasets
import torchvision.transforms as transforms

from torchsummary import summary

In [153]:
import torch
import torchvision 
from torchvision import transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((125.3/255, 123.0/255, 113.9/255), (63.0/255, 62.1/255.0, 66.7/255.0)),
])

testset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform)
testloaderIn = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=2)

Files already downloaded and verified


# 딥러닝 코딩 step by step

## step1. 데이터 전처리 
1. `sklearn.model_selection.train_test_split` 메소드를 이용하여 train, val, test 분리
     - 필요에 따라 `torch.utils.data.Dataset`을 상속(필수 overriding: `__init__`, `__len__`, `__getitem__`)하여 custom dataset을 만들어야 한다. 
2. `torch.utils.data.DataLoader`를 이용하여 mini batch 호출을 정의한다.
3. `model` 정의: 원하는 모델을 만들기(classifier든, detection이든, segmentation이든)

In [337]:
from torch.utils.data import Dataset

class svhnDataset(Dataset):
    def __init__(self, data, transform=None):
        super(svhnDataset, self).__init__()
        self.img_list = []
        self.label_list = []
        for data_pair in data:
            self.img_list.append(data_pair[0])
            self.label_list.append(data_pair[1])
            
        self.transform = transform
    
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, idx):
        image = self.img_list[idx]
        label = self.label_list[idx]
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [338]:
import torch
import torchvision 
from torchvision import transforms

path2data = '../data'

train_transform =transforms.Compose([
    transforms.RandomCrop(28, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4376821, 0.4437697, 0.47280442), (0.19803012, 0.20101562, 0.19703614)),
    transforms.Resize(32)
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4376821, 0.4437697, 0.47280442), (0.19803012, 0.20101562, 0.19703614)),
    transforms.Resize(32)
])

train_ds = datasets.SVHN(root=path2data, split='train', download=True)
test_ds = datasets.SVHN(root=path2data, split='test', download=True)

train_ds = svhnDataset(data=train_ds, transform=train_transform)
test_ds = svhnDataset(data=test_ds, transform=test_transform)

Using downloaded and verified file: ../data/train_32x32.mat
Using downloaded and verified file: ../data/test_32x32.mat


In [339]:
from torch.utils.data import DataLoader
BATCH_SIZE = 128

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last = False)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, drop_last = False)

In [353]:
path2model = '../models/'
model_name = 'densenet10.pth'
dense10_cifar = torch.load(path2model+model_name)

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute o

In [341]:
# check whether the model do inference well or not
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = torch.randn((3, 3, 32, 32)).to(device)
# check MODEL
model = dense10_cifar.to(device)
output_Stem = model(x)
print('Input size:', x.size())
print('Stem output size:', output_Stem.size())

Input size: torch.Size([3, 3, 32, 32])
Stem output size: torch.Size([3, 10])


pretrained model `densenet10.pth`가 pytorch 버전 문제인지 일부 기능이 안된다. `print(densenet10)`을 실행하면 `KeyError: 'track_running_stats`라는 error를 낸다.

In [354]:
# version 차이 때문인지, 아래와 같은 오류가 발생.
# AttributeError: 'BatchNorm2d' object has no attribute 'track_running_stats'`
# AttributeError: 'BatchNorm2d' object has no attribute 'num_batches_tracked'`
# 주어진 model 내의 BatchNorm2d module의 위의 속성들을 True로 바꿈으로써 해결 가능
def recursion_change_bn(module):
    if isinstance(module, torch.nn.BatchNorm2d):
        module.track_running_stats = True
        module.num_batches_tracked = True
    else:
        for i, (name, module1) in enumerate(module._modules.items()):
            module1 = recursion_change_bn(module1)
            
recursion_change_bn(dense10_cifar)

### `change_bn`의 작동 원리가 뭘까?
`module._modules`는 무엇일까?

In [343]:
def dn_recursive_change_bn(module):
    if isinstance(module, torch.nn.BatchNorm2d):
        module.track_running_stats = True
#         module.num_batches_tracked = True
    else:
        for i, (name, module1) in enumerate(module._modules.items()):
            module1 = recursion_change_bn(module1)
            
dn_recursive_change_bn(dense10_cifar)

In [305]:
len(dense10_cifar._modules)#.items())

9

### End of `change_bn`
---

In [356]:
import time

EPOCHS = 200
DEVICE = None

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"DEVICE : {DEVICE}")

model = dense10_cifar.to(DEVICE)
optimizer = torch.optim.SGD(dense10_cifar.parameters(), lr=0.01, momentum=0.5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)
criterion = torch.nn.CrossEntropyLoss(reduction='sum')

def train(model, train_loader, val_loader, topk=3):
    
    model = model.to(DEVICE)

    start = int(time.time())
    result_dict = {'EPOCH':[], 'Train Loss':[], 'Train Top 1 Acc':[], 'Train Top k Acc':[], 'Val Loss':[], 'Val Top 1 Acc':[], 'Val Top k Acc':[], 'Time':[]}
    for epoch in range(1, EPOCHS + 1):
        
        train_loss, train_top1_acc, train_topk_acc = _train(model, train_loader, topk=topk)
        scheduler.step()
        val_loss, val_top1_acc, val_topk_acc = _evaluate(model, val_loader, topk=topk)
        if epoch % 50 == 0:
            torch.save(model, '../models/densenet10_svhn_epoch'+str(epoch))
        cur_time = int(time.time())
        overtime = cur_time - start # the type of `time` is float
#         print(f'Epoch: {epoch:>3}/{EPOCHS}  Train Loss: {train_loss:.3f}  Train Top 1 Acc: {train_top1_acc:.2f}%  Val Loss: {val_loss:.3f}  Val Top 1 Acc: {val_top1_acc:.2f}%')
#         print(f'                                   Train Top 3 Acc: {train_topk_acc:.2f}%                   Val Top 3 Acc: {val_topk_acc:.2f}%   Overtime: {overtime//3600:>2}H {(overtime//60)%60:>2}M {overtime%60:>2}S')
        result_dict['EPOCH'].append(epoch)
        result_dict['Train Loss'].append(train_loss)
        result_dict['Train Top 1 Acc'].append(train_top1_acc)
        result_dict['Train Top k Acc'].append(train_topk_acc)
        result_dict['Val Loss'].append(val_loss)
        result_dict['Val Top 1 Acc'].append(val_top1_acc)
        result_dict['Val Top k Acc'].append(val_topk_acc)
        result_dict['Time'].append(overtime)
    return result_dict

def _train(model, train_loader, topk=3):
    model.train()
    train_loss = 0
    correct_top1 = 0
    correct_topk = 0
    
    for image, label in train_loader:
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()               # global `optimizer`(gredient descent method)
        pred_proba = model(image)
        loss = criterion(pred_proba, label) # global `criterion`(loss function)
        loss.backward()
        optimizer.step()
        # cummulative sum of loss
        train_loss += loss.item()
        # the number of correct top1 prediction
        top1_pred = pred_proba.max(1, keepdim=True)[1]
        correct_top1 += top1_pred.eq(label.view_as(top1_pred)).sum().item()
        # the number of correct topk prediction
        topk_pred = pred_proba.topk(k=topk, dim=1)[1] # len == 2인 tensor인데, index == 0에는 sorted topk proba가 있다.
        correct_topk += topk_pred.eq(label.view(-1, 1).expand_as(topk_pred)).sum().item()
        
    train_loss /= len(train_loader.dataset)
    top1_accuracy = 100. * correct_top1 / len(train_loader.dataset)
    topk_accuracy = 100. * correct_topk / len(train_loader.dataset)
    return train_loss, top1_accuracy, topk_accuracy

def _evaluate(model, val_loader, topk=3):
    model.eval() # using this, the dropouts turn off
    val_loss = 0
    correct_top1 = 0
    correct_topk = 0
    
    with torch.no_grad(): # turn of auto-tracking of gradient
        for image, label in val_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            pred_proba = model(image)
            # cummulative sum of loss
            val_loss += criterion(pred_proba, label).item()
            # the number of correct top1 prediction
            top1_pred = pred_proba.max(1, keepdim=True)[1]
            correct_top1 += top1_pred.eq(label.view_as(top1_pred)).sum().item()
            # the number of correct topk prediction
            topk_pred = pred_proba.topk(k=topk, dim=1)[1]
            correct_topk += topk_pred.eq(label.view(-1, 1).expand_as(topk_pred)).sum().item()
            
    val_loss /= len(val_loader.dataset)
    top1_accuracy = 100. * correct_top1 / len(val_loader.dataset)
    topk_accuracy = 100. * correct_topk / len(val_loader.dataset)
    return val_loss, top1_accuracy, topk_accuracy

DEVICE : cuda


### Start of Experiments
---
#### 아래 셀들은 model의 `callable 함수의 output`과 `loss 함수의 output` 이 어떤 것인지 확인하는 실험과 top-k error에 대한 실험이다.


In [250]:
DEVICE = 'cuda'
optimizer = torch.optim.SGD(dense10_cifar.parameters(), lr=0.01, momentum=0.5)
criterion1 = nn.CrossEntropyLoss()
criterion2 = nn.CrossEntropyLoss(reduction='sum')

dense10_cifar.train()
train_loss = 0
correct_top1 = 0
correct_top5 = 0

prediction = None
loss_variable1 = None
loss_varaibel2 = None
label = None

for batch_idx, (image, label) in enumerate(train_loader):
    image = image.to(DEVICE)
    label = label.to(DEVICE)
    optimizer.zero_grad()
    prediction = model(image)
    loss_variable1 = criterion1(prediction, label)
    loss_variable2 = criterion2(prediction, label)
    
    loss_variable1.backward()
    optimizer.step()
    
    break

##### 아래와 같은 방식으로 top 1 predictioin value에 접근 가능

In [166]:
pred = output.argmax(1, keepdim=True)
# 또는
pred = output.max(1, keepdim=True)[1]

tensor(18.8109, device='cuda:0', grad_fn=<MaxBackward1>)

model의 callable method의 output은 `torch.Tensor`이다. `torch.Tensor.max()`, `torch.Tensor.argmas()` 모두 첫번째 parameter는 `dim`이다. ~~이것이 무엇을 의미하는지 정확히 모르겠다.~~ `dim`은 `max`(또는 `argmax`)가 적용될 tensor의 차원의 index를 의미한다. 즉, 해당 차원 방향으로 max value 또는 max index를 찾는 것이다.<br>
두번째 parameter는 `keepdim`으로 `bool` type이다. default는 `False`이다. `True`이면 해당 input tensor를 그대로 유지한 채로 return한다.
`max()` mothod는 tuple을 return하는데, 첫번째 값에는 `tensro of max value`가, 두번째 값에는 `tensor of index of max value`가 저장된다.
<br><br>
Appendix: input의 형태가 [A,B,C,D]라고 할 때
dim=n 이라고 하면 n번째를 제외한 output이 나오게 되고
ex) dim = 2, C가 빠져 [A,B,D]가 나오게 된다.
이 C에 해당하는 데이터를 기준으로 최대값 및 인덱스가 튜플로 나오게 된다.
```
--------------------Example--------------------
INPUT : 
tensor([[[3, 4, 2, 0, 3],
         [0, 4, 0, 3, 0],
         [2, 0, 4, 3, 3],
         [1, 3, 3, 0, 4]],

        [[0, 3, 4, 2, 0],
         [0, 3, 4, 0, 2],
         [4, 0, 3, 2, 2],
         [3, 1, 1, 0, 1]],

        [[3, 4, 3, 0, 2],
         [2, 4, 1, 3, 1],
         [4, 0, 1, 1, 4],
         [4, 4, 1, 1, 0]]])
dim=2
torch.return_types.max(
values=tensor([[4, 4, 4, 4],
        [4, 4, 4, 3],
        [4, 4, 4, 4]]),
indices=tensor([[1, 1, 2, 4],
        [2, 2, 0, 0],
        [1, 1, 0, 0]]))
dim=1
torch.return_types.max(
values=tensor([[3, 4, 4, 3, 4],
        [4, 3, 4, 2, 2],
        [4, 4, 3, 3, 4]]),
indices=tensor([[0, 0, 2, 1, 3],
        [2, 0, 0, 0, 1],
        [2, 0, 0, 1, 2]]))
dim=0
torch.return_types.max(
values=tensor([[3, 4, 4, 2, 3],
        [2, 4, 4, 3, 2],
        [4, 0, 4, 3, 4],
        [4, 4, 3, 1, 4]]),
indices=tensor([[0, 0, 1, 1, 0],
        [2, 0, 1, 0, 1],
        [1, 0, 0, 0, 2],
        [2, 2, 0, 2, 0]]))
dim= -1
torch.return_types.max(
values=tensor([[4, 4, 4, 4],
        [4, 4, 4, 3],
        [4, 4, 4, 4]]),
indices=tensor([[1, 1, 2, 4],
        [2, 2, 0, 0],
        [1, 1, 0, 0]]))
```

#### 다음으로 loss function `criterion`의 output에 대해 알아보자

여전히 `torch.Tensor`이다. 다만, `grad_fn`이라는 속성이 다르다. 해당 속성이 무엇을 의미하는지는 아직 모르겠지만, loss function의 output tensor는 이 속성덕분에 `backward()`가 가능한 것 같다.

In [228]:
loss_variable1

7.324499130249023

In [229]:
loss_variable1.item() * 128

937.535888671875

In [230]:
loss_variable2.item()

937.535888671875

loss function은 default로 입력으로 받은 batch의 평균 loss를 반환한다(파라미터 중 하나인 `reduction`이 `reduction='mean'`이기때문). 미니배치 평균 loss들의 평균은 전체 batch의 평균 loss와 다르다. 따라서 전체 배치의 loss합산에 batch size(위 코드에서는 `len(train_loader.dataset)`로 구할 수 있음)로 나눠주는 것이 진정 평균 loss이다. 

#### About top-5 error

<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FKiVxY%2Fbtq4wvikM2O%2FwKfU8cCTahrZzQlsYuTk2k%2Fimg.png'>

In [255]:
prediction[0:2]

tensor([[-0.8047, -2.2865,  1.6786, -1.5653,  3.0456, -2.6758,  6.6618, -1.9233,
         -5.7986,  3.6604],
        [-3.0336,  1.3451,  0.8680, -0.2392,  2.8278, -2.3174,  1.4950, -3.9047,
         -2.1566,  5.1090]], device='cuda:0', grad_fn=<SliceBackward>)

In [280]:
topk_pred = prediction[0:2].topk(k=7, dim=1)
display(topk_pred)

torch.return_types.topk(values=tensor([[ 6.6618,  3.6604,  3.0456,  1.6786, -0.8047, -1.5653, -1.9233],
        [ 5.1090,  2.8278,  1.4950,  1.3451,  0.8680, -0.2392, -2.1566]],
       device='cuda:0', grad_fn=<TopkBackward>), indices=tensor([[6, 9, 4, 2, 0, 3, 7],
        [9, 4, 6, 1, 2, 3, 8]], device='cuda:0'))

In [271]:
topk_pred.eq(label[0:2].view(-1, 1).expand_as(topk_pred))

tensor([[False, False, False, False, False,  True, False],
        [False, False, False, False, False, False,  True]], device='cuda:0')

In [273]:
topk_pred.eq(label[0:2].view(-1, 1).expand_as(topk_pred)).sum()

tensor(2, device='cuda:0')

In [274]:
topk_pred.eq(label[0:2].view(-1, 1).expand_as(topk_pred)).sum().item()

2

### End of Experiments
---

In [ ]:
result_dict = train(model, train_loader, test_loader)

In [373]:
import pandas as pd
pd.DataFrame(result_dict).to_csv('dense10_svhn_training_history.csv', index=False)
result_df = pd.read_csv('dense10_svhn_training_history.csv')
display(result_df)

,EPOCH,Train Loss,Train Top 1 Acc,Train Top k Acc,Val Loss,Val Top 1 Acc,Val Top k Acc,Time
0,1,2.273483,18.578429,44.375282,2.226622,19.587431,46.550400,81
1,2,2.128706,23.287877,50.123538,1.888511,26.720959,63.929010,163
2,3,1.095759,62.746222,86.348062,1.119816,69.149508,87.234942,246
3,4,0.539728,83.174304,95.013446,0.723782,76.329133,93.853719,328
4,5,0.446809,86.014988,95.951240,0.532208,83.090043,95.440227,411
...,...,...,...,...,...,...,...,...
195,196,0.038833,98.671799,99.931747,0.316018,93.696220,98.413491,16184
196,197,0.037876,98.716846,99.930382,0.305350,93.980486,98.413491,16267
197,198,0.037555,98.749608,99.916732,0.319139,93.607867,98.436540,16349
198,199,0.036705,98.775544,99.923557,0.319001,93.788414,98.340504,16431


In [367]:
for i in range(200):
    result_sr = result_df.iloc[i]
    epoch = int(result_sr['EPOCH'])
    train_loss = result_sr['Train Loss']
    train_top1_acc = result_sr['Train Top 1 Acc']
    train_topk_acc = result_sr['Train Top k Acc']
    val_loss = result_sr['Val Loss']
    val_top1_acc = result_sr['Val Top 1 Acc']
    val_topk_acc = result_sr['Val Top k Acc']
    overtime = int(result_sr['Time'])
    print(f'Epoch: {epoch:>3}/{EPOCHS}  Train Loss: {train_loss:.3f}  Train Top 1 Acc: {train_top1_acc:.2f}%  Val Loss: {val_loss:.3f}  Val Top 1 Acc: {val_top1_acc:.2f}%')
    print(f'                                   Train Top 3 Acc: {train_topk_acc:.2f}%                   Val Top 3 Acc: {val_topk_acc:.2f}%   Overtime: {overtime//3600:>2}H {(overtime//60)%60:>2}M {overtime%60:>2}S')

Epoch:   1/200  Train Loss: 2.273  Train Top 1 Acc: 18.58%  Val Loss: 2.227  Val Top 1 Acc: 19.59%
                                   Train Top 3 Acc: 44.38%                   Val Top 3 Acc: 46.55%   Overtime:  0H  1M 21S
Epoch:   2/200  Train Loss: 2.129  Train Top 1 Acc: 23.29%  Val Loss: 1.889  Val Top 1 Acc: 26.72%
                                   Train Top 3 Acc: 50.12%                   Val Top 3 Acc: 63.93%   Overtime:  0H  2M 43S
Epoch:   3/200  Train Loss: 1.096  Train Top 1 Acc: 62.75%  Val Loss: 1.120  Val Top 1 Acc: 69.15%
                                   Train Top 3 Acc: 86.35%                   Val Top 3 Acc: 87.23%   Overtime:  0H  4M  6S
Epoch:   4/200  Train Loss: 0.540  Train Top 1 Acc: 83.17%  Val Loss: 0.724  Val Top 1 Acc: 76.33%
                                   Train Top 3 Acc: 95.01%                   Val Top 3 Acc: 93.85%   Overtime:  0H  5M 28S
Epoch:   5/200  Train Loss: 0.447  Train Top 1 Acc: 86.01%  Val Loss: 0.532  Val Top 1 Acc: 83.09%
             

In [374]:
dense10_svhn = torch.load('../models/densenet10_svhn.pth')

In [375]:
_evaluate(model=dense10_svhn,topk=3, val_loader=test_loader)

(0.31109534664687455, 94.07652120467118, 98.50568531038722)

In [389]:
'''cal.py copied'''
from __future__ import print_function
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import time
from scipy import misc
import calMetric as m
import calData as d
#CUDA_DEVICE = 0

start = time.time()
#loading data sets

transform = transforms.Compose([
    transforms.ToTensor(),
    # `Normalize` is the class which normalize data channelwise by the given mean and std.
    # In this case, the number of input channels is 3 because the lenght of the iterable object is 3.
    # Ref: https://pytorch.org/vision/stable/transforms.html
    # Ref of various image normalizing: https://stackoverflow.com/questions/33610825/normalization-in-image-processing
    transforms.Normalize((0.4376821, 0.4437697, 0.47280442), (0.19803012, 0.20101562, 0.19703614)),
    transforms.Resize(32)
])




# loading neural network

# Name of neural networks
# Densenet trained on CIFAR-10:         densenet10
# Densenet trained on CIFAR-100:        densenet100
# Densenet trained on WideResNet-10:    wideresnet10
# Densenet trained on WideResNet-100:   wideresnet100
#nnName = "densenet10"

#imName = "Imagenet"



criterion = nn.CrossEntropyLoss()



def test(nnName, dataName, CUDA_DEVICE, epsilon, temperature):
    
    net1 = torch.load("../models/{}.pth".format(nnName))
    optimizer1 = optim.SGD(net1.parameters(), lr = 0, momentum = 0)
    net1.cuda(CUDA_DEVICE)
    
    # We suppose the out-of distribution data is `Uniform` or `Gaussian`. 
    # If you do not, we call the data as `testsetout`.
    if dataName != "Uniform" and dataName != "Gaussian":
        testsetout = torchvision.datasets.ImageFolder("../data/{}".format(dataName), transform=transform)
        testloaderOut = torch.utils.data.DataLoader(testsetout, batch_size=1,
                                         shuffle=False, num_workers=2)
        
    # OOD 성능을 평가할 때는 고정되어야 하는 것들이 있다.
    # 적어도 같은 모델이어야 하고 같은 데이터이어야 한다(in dist & out-of dist 모두). (이것도 하이퍼 파라미터라고 볼 수 있을까?)
    # 그래야 OOD 방법론에 대한 정량적인 평가가 가능하기때문.
    if nnName == "densenet10" or nnName == "wideresnet10": 
        testset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform)
        testloaderIn = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False, num_workers=2)
        
    if nnName == "densenet100" or nnName == "wideresnet100": 
        testset = torchvision.datasets.CIFAR100(root='../data', train=False, download=True, transform=transform)
        testloaderIn = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False, num_workers=2)
    if nnName == "densenet10_svhn":
        testset = torchvision.datasets.SVHN(root='../data', split='test', download=True, transform=transform)
        testloaderIn = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False, num_workers=2)
    
    if dataName == "Gaussian":
        d.testGaussian(net1, criterion, CUDA_DEVICE, testloaderIn, testloaderIn, nnName, dataName, epsilon, temperature)
        m.metric(nnName, dataName)

    elif dataName == "Uniform":
        d.testUni(net1, criterion, CUDA_DEVICE, testloaderIn, testloaderIn, nnName, dataName, epsilon, temperature)
        m.metric(nnName, dataName)
        
    else:
        d.testData(net1, criterion, CUDA_DEVICE, testloaderIn, testloaderOut, nnName, dataName, epsilon, temperature) 
        m.metric(nnName, dataName)


In [6]:
import  cal as c
c.test('densenet10_svhn', 'Imagenet', 0, 0.0014, 1000) # 첫번째 인자는 모델명, 두번째 인자는 ood dataset의 이름, 세번째는 # gpu

Using downloaded and verified file: ../data/test_32x32.mat
Processing in-distribution images


/home/mydir/dn2ood/odin/code/calData.py:72: UserWarning: This overload of add is deprecated:
	add(Tensor input, Number alpha, Tensor other, *, Tensor out)
Consider using one of the following signatures instead:
	add(Tensor input, Tensor other, *, Number alpha, Tensor out) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1025.)
  tempInputs = torch.add(inputs.data,  -noiseMagnitude1, gradient)


 100/9000 images processed, 4.8 seconds used.
 200/9000 images processed, 4.0 seconds used.
 300/9000 images processed, 4.0 seconds used.
 400/9000 images processed, 4.0 seconds used.
 500/9000 images processed, 4.0 seconds used.
 600/9000 images processed, 4.0 seconds used.
 700/9000 images processed, 4.0 seconds used.
 800/9000 images processed, 4.0 seconds used.
 900/9000 images processed, 4.0 seconds used.
1000/9000 images processed, 4.0 seconds used.
1100/9000 images processed, 4.0 seconds used.
1200/9000 images processed, 4.0 seconds used.
1300/9000 images processed, 4.0 seconds used.
1400/9000 images processed, 4.0 seconds used.
1500/9000 images processed, 4.0 seconds used.
1600/9000 images processed, 4.0 seconds used.
1700/9000 images processed, 4.0 seconds used.
1800/9000 images processed, 4.0 seconds used.
1900/9000 images processed, 4.0 seconds used.
2000/9000 images processed, 4.0 seconds used.
2100/9000 images processed, 4.0 seconds used.
2200/9000 images processed, 4.0 se

In [7]:
c.test('densenet10', 'Imagenet_crop', 0, 0.0014, 1000)

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute o

Files already downloaded and verified
Processing in-distribution images
 100/9000 images processed, 4.7 seconds used.
 200/9000 images processed, 4.0 seconds used.
 300/9000 images processed, 4.0 seconds used.
 400/9000 images processed, 4.0 seconds used.
 500/9000 images processed, 4.1 seconds used.
 600/9000 images processed, 4.2 seconds used.
 700/9000 images processed, 4.2 seconds used.
 800/9000 images processed, 4.2 seconds used.
 900/9000 images processed, 4.1 seconds used.
1000/9000 images processed, 4.1 seconds used.
1100/9000 images processed, 4.1 seconds used.
1200/9000 images processed, 4.1 seconds used.
1300/9000 images processed, 4.2 seconds used.
1400/9000 images processed, 4.2 seconds used.
1500/9000 images processed, 4.2 seconds used.
1600/9000 images processed, 4.2 seconds used.
1700/9000 images processed, 4.0 seconds used.
1800/9000 images processed, 4.0 seconds used.
1900/9000 images processed, 4.0 seconds used.
2000/9000 images processed, 4.0 seconds used.
2100/900

In [9]:
c.test('densenet10_svhn', 'Imagenet_resize', 0, 0.0014, 1000)

Using downloaded and verified file: ../data/test_32x32.mat
Processing in-distribution images
 100/9000 images processed, 4.8 seconds used.
 200/9000 images processed, 4.0 seconds used.
 300/9000 images processed, 4.0 seconds used.
 400/9000 images processed, 4.0 seconds used.
 500/9000 images processed, 4.0 seconds used.
 600/9000 images processed, 4.0 seconds used.
 700/9000 images processed, 4.0 seconds used.
 800/9000 images processed, 4.0 seconds used.
 900/9000 images processed, 4.0 seconds used.
1000/9000 images processed, 4.0 seconds used.
1100/9000 images processed, 4.0 seconds used.
1200/9000 images processed, 4.0 seconds used.
1300/9000 images processed, 4.0 seconds used.
1400/9000 images processed, 4.0 seconds used.
1500/9000 images processed, 4.0 seconds used.
1600/9000 images processed, 4.0 seconds used.
1700/9000 images processed, 4.0 seconds used.
1800/9000 images processed, 4.0 seconds used.
1900/9000 images processed, 4.0 seconds used.
2000/9000 images processed, 4.0 s

In [11]:
c.test('densenet10_svhn', 'LSUN_crop', 0, 0.0014, 1000)

Using downloaded and verified file: ../data/test_32x32.mat
Processing in-distribution images
 100/9000 images processed, 4.8 seconds used.
 200/9000 images processed, 4.0 seconds used.
 300/9000 images processed, 4.0 seconds used.
 400/9000 images processed, 4.0 seconds used.
 500/9000 images processed, 4.0 seconds used.
 600/9000 images processed, 4.0 seconds used.
 700/9000 images processed, 4.0 seconds used.
 800/9000 images processed, 4.0 seconds used.
 900/9000 images processed, 4.0 seconds used.
1000/9000 images processed, 4.0 seconds used.
1100/9000 images processed, 4.0 seconds used.
1200/9000 images processed, 4.0 seconds used.
1300/9000 images processed, 4.0 seconds used.
1400/9000 images processed, 4.0 seconds used.
1500/9000 images processed, 4.0 seconds used.
1600/9000 images processed, 4.0 seconds used.
1700/9000 images processed, 4.0 seconds used.
1800/9000 images processed, 4.0 seconds used.
1900/9000 images processed, 4.0 seconds used.
2000/9000 images processed, 4.0 s

UnboundLocalError: local variable 'dataName' referenced before assignment

In [1]:
import calMetric as m
m.metric('densenet10_svhn', 'LSUN_crop')

Neural network architecture:          DenseNet-BC-100
In-distribution dataset:                         SVHN
Out-of-distribution dataset:              LSUN (crop)

                          Baseline         Our Method
FPR at TPR 95%:              41.2%              31.1% 
Detection error:             13.2%              14.0%
AUROC:                       93.2%              92.9%
AUPR In:                     93.4%              92.1%
AUPR Out:                    91.8%              93.2%


In [3]:
import  cal as c
c.test('densenet10_svhn', 'LSUN_resize', 0, 0.0014, 1000) 

Using downloaded and verified file: ../data/test_32x32.mat
Processing in-distribution images


/home/mydir/dn2ood/odin/code/calData.py:72: UserWarning: This overload of add is deprecated:
	add(Tensor input, Number alpha, Tensor other, *, Tensor out)
Consider using one of the following signatures instead:
	add(Tensor input, Tensor other, *, Number alpha, Tensor out) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1025.)
  tempInputs = torch.add(inputs.data,  -noiseMagnitude1, gradient)


 100/9000 images processed, 4.9 seconds used.
 200/9000 images processed, 4.0 seconds used.
 300/9000 images processed, 3.9 seconds used.
 400/9000 images processed, 4.0 seconds used.
 500/9000 images processed, 4.0 seconds used.
 600/9000 images processed, 4.0 seconds used.
 700/9000 images processed, 4.0 seconds used.
 800/9000 images processed, 4.0 seconds used.
 900/9000 images processed, 4.0 seconds used.
1000/9000 images processed, 3.9 seconds used.
1100/9000 images processed, 4.0 seconds used.
1200/9000 images processed, 4.0 seconds used.
1300/9000 images processed, 4.0 seconds used.
1400/9000 images processed, 4.0 seconds used.
1500/9000 images processed, 4.0 seconds used.
1600/9000 images processed, 4.0 seconds used.
1700/9000 images processed, 3.9 seconds used.
1800/9000 images processed, 4.0 seconds used.
1900/9000 images processed, 4.0 seconds used.
2000/9000 images processed, 4.0 seconds used.
2100/9000 images processed, 4.0 seconds used.
2200/9000 images processed, 4.0 se

In [4]:
c.test('densenet10_svhn', 'iSUN', 0, 0.0014, 1000) 

Using downloaded and verified file: ../data/test_32x32.mat
Processing in-distribution images
 100/7925 images processed, 4.8 seconds used.
 200/7925 images processed, 4.0 seconds used.
 300/7925 images processed, 4.0 seconds used.
 400/7925 images processed, 4.0 seconds used.
 500/7925 images processed, 4.0 seconds used.
 600/7925 images processed, 4.0 seconds used.
 700/7925 images processed, 4.0 seconds used.
 800/7925 images processed, 4.0 seconds used.
 900/7925 images processed, 4.0 seconds used.
1000/7925 images processed, 4.0 seconds used.
1100/7925 images processed, 4.0 seconds used.
1200/7925 images processed, 4.0 seconds used.
1300/7925 images processed, 4.0 seconds used.
1400/7925 images processed, 4.0 seconds used.
1500/7925 images processed, 4.0 seconds used.
1600/7925 images processed, 4.0 seconds used.
1700/7925 images processed, 4.0 seconds used.
1800/7925 images processed, 4.0 seconds used.
1900/7925 images processed, 4.0 seconds used.
2000/7925 images processed, 4.0 s

In [5]:
c.test('densenet10_svhn', 'Gaussian', 0, 0.0014, 1000) 

Using downloaded and verified file: ../data/test_32x32.mat
Processing in-distribution images
 100/9000 images processed, 4.7 seconds used.
 200/9000 images processed, 4.0 seconds used.
 300/9000 images processed, 4.0 seconds used.
 400/9000 images processed, 4.0 seconds used.
 500/9000 images processed, 4.0 seconds used.
 600/9000 images processed, 4.0 seconds used.
 700/9000 images processed, 4.0 seconds used.
 800/9000 images processed, 4.0 seconds used.
 900/9000 images processed, 4.0 seconds used.
1000/9000 images processed, 4.0 seconds used.
1100/9000 images processed, 4.0 seconds used.
1200/9000 images processed, 4.0 seconds used.
1300/9000 images processed, 4.0 seconds used.
1400/9000 images processed, 4.0 seconds used.
1500/9000 images processed, 4.0 seconds used.
1600/9000 images processed, 4.0 seconds used.
1700/9000 images processed, 4.0 seconds used.
1800/9000 images processed, 4.1 seconds used.
1900/9000 images processed, 4.2 seconds used.
2000/9000 images processed, 4.2 s

In [6]:
c.test('densenet10_svhn', 'Uniform', 0, 0.0014, 1000) 

Using downloaded and verified file: ../data/test_32x32.mat
Processing in-distribution images
 100/9000  images processed, 4.8 seconds used.
 200/9000  images processed, 4.0 seconds used.
 300/9000  images processed, 4.0 seconds used.
 400/9000  images processed, 4.0 seconds used.
 500/9000  images processed, 4.1 seconds used.
 600/9000  images processed, 4.1 seconds used.
 700/9000  images processed, 4.2 seconds used.
 800/9000  images processed, 4.1 seconds used.
 900/9000  images processed, 4.0 seconds used.
1000/9000  images processed, 4.0 seconds used.
1100/9000  images processed, 4.0 seconds used.
1200/9000  images processed, 4.0 seconds used.
1300/9000  images processed, 4.0 seconds used.
1400/9000  images processed, 4.0 seconds used.
1500/9000  images processed, 4.2 seconds used.
1600/9000  images processed, 4.3 seconds used.
1700/9000  images processed, 4.2 seconds used.
1800/9000  images processed, 4.1 seconds used.
1900/9000  images processed, 4.3 seconds used.
2000/9000  ima